In [3]:
import pandas as pd
import sqlite3
import os
import shutil
import tarfile
import re
import numpy as np
import os
import json
import re
import pickle
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import multiprocessing
import nltk
from gensim.scripts.glove2word2vec import glove2word2vec
import gensim.downloader as api
import gzip
import gzip
import requests
from gensim.models import KeyedVectors
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
import re

## Tweet Preprocessing script

In [ ]:
def camel_case_split(identifier):
    # https://stackoverflow.com/a/29920015/2799941
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return [m.group(0) for m in matches]


def process_hashtag(ht):
    ht_body = ht.group()[1:]
    if ht_body.upper() == ht_body:
        return "<HASHTAG> {ht_body} <ALLCAPS>"
    return "<HASHTAG> " + " ".join(camel_case_split(ht_body))


def remove_emoji(tweet):
    """"
    Utility function to remove emojis from tweet text using simple regex statements..
    """
    emoji_pattern = re.compile("[" u"\U0001f680-\U0001f6FF"
                               u"\U0001f300-\U0001f64F"
                               u"\U00002600-\U000027BF"
                               u"\U00002700-\U000027BF"
                               u"\U00002600-\U000026FF"
                               u"\U0001f600-\U0001f64F"
                               u"\U0001f300-\U0001f5fF"
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U0001F914"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', tweet)  # no emoji
    return text


def remove_punctuation(tweet):
    """
    Utility function to remove punctuations from tweet text using simple regex statements..
    """

    text = re.sub(r'(?<=\w)[^\s\w\'\>](?![^\s\w])', '', tweet)
    # text = re.sub(r'[^\w\s]', ' ', tweet)
    return text


def preprocess(tweet_text):
    """ Run raw tweet JSON through preprocessing/cleaning steps.
        Adapted from Stanford Glove site's Ruby tweet processing script.
        See: https://nlp.stanford.edu/projects/glove/preprocess-twitter.rb
    """
    eyes = r"[8:=;]"
    nose = r"['`\-]?"
    # remove urls
    url = lambda tweet: re.sub(r"https?://\S+\b|www\.(\w+\.)+\S*", "<URL>", tweet)
    # remove newline characters \n
    newline = lambda tweet: re.sub(r"\n", "", tweet)
    # replace @ mentions with marker
    user = lambda tweet: re.sub(r"@\w+", "<USER>", tweet)
    # replace emojis with markers
    smile = lambda tweet: re.sub("{eyes}{nose}[)d]+|[)d]+{nose}{eyes}", "<SMILE>", tweet, re.I)
    lolface = lambda tweet: re.sub(r"{eyes}{nose}p+", "<LOLFACE>", tweet, re.I)
    sadface = lambda tweet: re.sub(r"{eyes}{nose}\(+|\)+{nose}{eyes}", "<SADFACE>", tweet)
    neutralface = lambda tweet: re.sub(r"{eyes}{nose}[\\/|l]", "<NEUTRALFACE>", tweet)
    # force slash-combined tokens to be two tokens
    slash = lambda tweet: re.sub(r"/", " / ", tweet)
    heart = lambda tweet: re.sub("[<|&lt;]3", "<HEART>", tweet)
    # replace &amp; with "and" plus <AMP> marker
    amp = lambda tweet: re.sub("&amp;", "and <AMP>", tweet)
    # replace year with marker (not very clean...)
    year = lambda tweet: re.sub("(?:17|18|19|20)\d{2}", "<YEAR>", tweet)
    # replace time with marker
    time = lambda tweet: re.sub("\d{1,2}:\d{2}(\s?)(am|pm)?", r"<TIME>\1", tweet)
    # replace numbers with marker
    number = lambda tweet: re.sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", "<NUMBER>", tweet)
    # add hashtag markers and indicator of whether all-caps or not
    # also split hashtags if joined capitalized words: BlackLivesMatter -> Black Lives Matter
    hashtag = lambda tweet: re.sub(r"#\S+", process_hashtag, tweet)
    # replace repeated characters (!!!) with single character and <REPEAT> marker
    repeat = lambda tweet: re.sub("([!?.]){2,}", r"\1 <REPEAT>", tweet)
    # replace elongated words (wayyyy) with original word and <ELONG> marker
    elong = lambda tweet: re.sub(r"\b(\S*?)(.)\2{2,}\b", r"\1\2 <ELONG>", tweet)
    # replace double spaces with single space
    spaces = lambda tweet: re.sub("\s+", " ", tweet)

    parse_funcs = [url, newline, user, smile,
                   lolface, sadface, neutralface, slash, heart,
                   amp, year, time, number, hashtag, repeat,
                   elong, spaces]

    for f in parse_funcs:
        tweet_text = f(tweet_text)

    tweet_text = remove_emoji(tweet_text)
    tweet_text = remove_punctuation(tweet_text)

    return tweet_text

# Reading tweet files and appending to tweet list

In [3]:
def read_tweet_files(directory_name, tweet_list):
    print("before", len(tweet_list))
    # fileDir = os.path.dirname(os.path.realpath('__file__'))
    print('dir', directory_name)
    # path = os.path.join(fileDir + directory_name)
    # print('path', path)

    good_tweets = 0
    bad_tweets = 0

    for file in os.listdir(directory_name):
        if not file.startswith('.'):
            date = file[:-4]
            print(date)
            with open(directory_name + '/' + file, 'r') as f:
                tweets = f.readlines()

            for tweetStr in tweets:
                try:
                    tweet = json.loads(tweetStr.strip().lower())
                    if 'body' in tweet:
                        text_key = 'body'
                    elif 'text' in tweet:
                        text_key = 'text'
                    tweet_text = preprocess(tweet[text_key])
                    if 'rt' not in tweet_text:
                        #print(tweet_text)
                        # tweet_text_eng = " ".join(w for w in nltk.wordpunct_tokenize(tweet_text) if w.lower() in words)
                        tweet_list.append(tweet_text.split())
                        good_tweets += 1
                except:
                    bad_tweets += 1

    print(good_tweets, bad_tweets)
    print("after", len(tweet_list))
    return tweet_list


In [3]:
def read_gzip_tweet_files(directory_name, tweet_list):
    print("before", len(tweet_list))
    # fileDir = os.path.dirname(os.path.realpath('__file__'))
    print('dir', directory_name)
    # path = os.path.join(fileDir + directory_name)
    # print('path', path)

    good_tweets = 0
    bad_tweets = 0

    for file in os.listdir(directory_name):
        if not file.startswith('.'):
            with gzip.open(directory_name + '/' + file, 'rb') as f:
                tweets = f.readlines()
                print(file, len(tweets))

            for tweetStr in tweets:
                try:
                    tweet = json.loads(tweetStr.strip().lower())
                    print(tweet)
                    if 'body' in tweet:
                        text_key = 'body'
                    elif 'text' in tweet:
                        text_key = 'text'
                    tweet_text = preprocess(tweet[text_key])
                    if 'rt' not in tweet_text:
                        print(tweet_text)
                        #tweet_text_ = " ".join(w for w in nltk.wordpunct_tokenize(tweet_text) if w.alpha())
                        # tweet_text_eng = " ".join(w for w in nltk.wordpunct_tokenize(tweet_text) if w.lower() in words)
                        tweet_list.append(tweet_text.split())
                        good_tweets += 1
                except:
                    bad_tweets += 1

    print(good_tweets, bad_tweets)
    print("after", len(tweet_list))
    return tweet_list


## Training word embeddings using gensim library

In [13]:
def build_model(tweetlist, title):
    model = Word2Vec(tweetlist,
                size=300,
                window=10,
                min_count=5,
                workers=multiprocessing.cpu_count())
    model.wv.save_word2vec_format(str(title)+'.txt', binary=False)
    #save_word2vec_format


## Utility functions to pickle and unpickle the lists

In [12]:
def pickle_list(tweets, title):
    with open(title+'.pkl', 'wb') as f:
        pickle.dump(tweets, f)


def unpickle_list(pickled_list):
    with open(pickled_list, 'rb') as f:
        unpickled_list = pickle.load(f)
    return unpickled_list


In [16]:
def get_glove_embedding(glovefile):
    embeddings_index = dict()
    f = api.load(glovefile)
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

## Comparison between word similarities in Depressed vs General Language

In [27]:
def word_similarity(word, model1, model2, model3):
    result1 = model1.wv.similar_by_word(word, 20)
    result2 = model2.wv.similar_by_word(word, 20)
    result3 = model3.wv.similar_by_word(word, 20)

    width = 20
    print("word: ",word,'\n')
    print("{} {} {}".format('healthy'.ljust(40), 'glove'.ljust(40), 'depressed'.ljust(width)))
    print("-----------------------------------------------------------------------------------------------------------")
    for i, r in enumerate(result1):
        print("{} {} | {} {} | {} {}".format(
            result1[i][0].ljust(width - 5),
            str(result1[i][1]).ljust(width),
            result2[i][0].ljust(width),
            str(result2[i][1]).ljust(width-8),
            result3[i][0].ljust(width),
            str(result3[i][1]).ljust(width - 5)))
    print('\n')


## Code for training incremental word embeddings for contextual shifts over the years

In [18]:
def makefolders(keyword, directory):
    try:
        os.mkdir(directory+keyword)
    except OSError:
        pass

In [19]:
def divide_files(directory):
    fileDir = os.path.dirname(os.path.realpath('__file__'))
    print(fileDir)

    for file in os.listdir(directory):
        if not file.startswith('.') and file[-3:] == '.gz':
            keyword = file.split('-')[0]
    
            #print(os.path.join(directory, file))
            if not os.path.exists(keyword):
                #pass
                makefolders(keyword, directory)
                shutil.move(os.path.join(directory, file), directory+keyword)
            else:
                shutil.move(os.path.join(directory, file), directory+keyword)

In [ ]:
#divide_files("/users/s/a/sandhya/twitter_scrap/keyword-search/raw-tweets/healthy_group_sampled/")

In [ ]:
def yearly_lists(d1, d2, years):
    for year in years:
        depressed_tweets = []
        depressed_tweets = read_tweet_files(d1+year, depressed_tweets)
        depressed_tweets = read_tweet_files(d2+year, depressed_tweets)
        pickle_list(depressed_tweets, 'depress_tweets'+'_'+str(year))


In [7]:
years = ['2012','2013','2014','2015','2016','2017','2018','2019']
#yearly_lists(depression_dir, depressed_dir, years)

In [14]:
def build_incremental_model(tweets, title, pretrained_vectors):
    model = Word2Vec(tweets,
                size=300,
                window=10,
                min_count=5,
                workers=multiprocessing.cpu_count())
    
    total_examples = model.corpus_count

    pretrained = KeyedVectors.load_word2vec_format(pretrained_vectors, binary=False, unicode_errors='ignore')
    model.build_vocab([list(pretrained.wv.vocab.keys())], update=True)
    model.intersect_word2vec_format(pretrained_vectors, binary=False, lockf=1.0, unicode_errors='ignore')
    
    model.train(tweets, total_examples=total_examples, epochs=model.iter)
    model.wv.save_word2vec_format(str(title)+'.txt', binary=False)
    #model.save(str(title)+'.bin')

In [37]:
def build_yearly_model(year):
    if year == '2012':
        tweets = unpickle_list('new_yearly_embeddings/cum_depress_tweets_'+year+'.pkl')
        print("count of tweets in year"+year,len(tweets))
        build_model(tweets, 'cum_depress_tweets_'+year)
    else:
        tweets = unpickle_list('new_yearly_embeddings/cum_depress_tweets_'+year+'.pkl') 
        print("count of tweets in year"+year,len(tweets))
        build_incremental_model(tweets, 'incremental_depress_tweets_'+year, 'yearly_embedding_files/depress_tweets_'+year+'.bin')
    
    

In [51]:
def remove_unicode(tweets):
    new_tweets = []
    for tweet in tweets:
        temp = []
        for word in tweet:
            word = regex.sub('', word).strip()
            if len(word) > 0:
                temp.append(word)
        new_tweets.append(temp)
    return new_tweets

In [27]:
new_tweets = []
for tweet in tweets:
    temp = []
    for word in tweet:
        word = regex.sub('', word).strip()
        if len(word) > 0:
            temp.append(word)
    new_tweets.append(temp)

In [52]:
tweets = remove_unicode(tweets)

In [1]:
years = ['2012','2013','2014','2015','2016','2017','2018','2019']
for year in years:
    build_yearly_model(year)